In [1]:
from qiskit import IBMQ
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.10.0',
 'qiskit-aer': '0.3.2',
 'qiskit-ignis': '0.2.0',
 'qiskit-ibmq-provider': '0.3.3',
 'qiskit-aqua': '0.6.1',
 'qiskit': '0.13.0'}

In [2]:
provider = IBMQ.load_account()
#change the provider based on your Qiskit account
#provider = IBMQ.get_provider(hub='ibm-q-ncsu', group='nc-state', project='on-boarding')
#if you do not have access to the twenty qubit machine, use this publically available ibmqx2 machine instead
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [3]:
from qiskit import *
from qiskit.compiler import transpile
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.aer import noise
import math

#Include our assertion function
from qiskit.compiler.assertion import classical_assertion, superposition_assertion, entanglement_assertion, calcSuccessrate

In [4]:
#we used ibmq_20_tokyo for our experiment, however, that backend has retired. So we can use the ibmqx2 instead
device = provider.get_backend('ibmqx2')
backend_monitor(device)
device

ibmqx2
Configuration
-------------
    n_qubits: 5
    operational: True
    status_msg: active
    pending_jobs: 79
    backend_version: 2.0.2
    basis_gates: ['u1', 'u2', 'u3', 'cx', 'id']
    local: False
    simulator: False
    backend_name: ibmqx2
    allow_object_storage: True
    max_experiments: 75
    quantum_volume: 8
    allow_q_circuit: False
    n_registers: 1
    online_date: 2017-01-24T05:00:00+00:00
    coupling_map: [[0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1], [2, 3], [2, 4], [3, 2], [3, 4], [4, 2], [4, 3]]
    open_pulse: False
    max_shots: 8192
    credits_required: True
    description: 5 qubit device
    url: None
    conditional: False
    memory: True
    sample_name: sparrow
    allow_q_object: True

Qubits [Name / Freq / T1 / T2 / U1 err / U2 err / U3 err / Readout err]
-----------------------------------------------------------------------
    Q0 / 5.28627 GHz / 50.31548 µs / 53.02446 µs / 0.0 / 0.00074 / 0.00147 / 0.019
    Q1 / 5.2379 GHz / 73.73208 

<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>

In [5]:
transQcList = []

pi = math.pi

In [6]:
def adder():
    first = "1"
    second = "1"

    l = len(first)
    l2 = len(second)
    if l > l2:
         n = l
    else:
         n = l2
    #Initializing the registers; two quantum registers with n bits each
    #1 more with n+1 bits, which will also hold the sum of the two #numbers
    #The classical register has n+1 bits, which is used to make the sum #readable
    a = QuantumRegister(n) #First number
    b = QuantumRegister(n+1) #Second number, then sum
    c = QuantumRegister(n) #Carry bits
    cl = ClassicalRegister(n+1) #Classical output
    #Combining all of them into one quantum circuit
    qc = QuantumCircuit(a, b, c, cl)

    #Setting up the registers using the values inputted
    for i in range(l):
        if first[i] == "1":
           qc.x(a[l - (i+1)]) #Flip the qubit from 0 to 1
    for i in range(l2):
       if second[i] == "1":
          qc.x(b[l2 - (i+1)]) #Flip the qubit from 0 to 1

    #Implementing a carry gate that is applied on all (c[i], a[i], b[i]) #with output fed to c[i+1]
    for i in range(n-1):
        qc.ccx(a[i], b[i], c[i+1])
        qc.cx(a[i], b[i])
        qc.ccx(c[i], b[i], c[i+1])
    #For the last iteration of the carry gate, instead of feeding the #result to c[n], we use b[n], which is why c has only n bits, with #c[n-1] being the last carry bit
    qc.ccx(a[n-1], b[n-1], b[n])
    qc.cx(a[n-1], b[n-1])
    qc.ccx(c[n-1], b[n-1], b[n])

    #Reversing the gate operation performed on b[n-1]
    qc.cx(c[n-1], b[n-1])
    #Reversing the gate operations performed during the carry gate implementations
    #This is done to ensure the sum gates are fed with the correct input bit states
    for i in range(n-1):
        qc.ccx(c[(n-2)-i], b[(n-2)-i], c[(n-1)-i])
        qc.cx(a[(n-2)-i], b[(n-2)-i])
        qc.ccx(a[(n-2)-i], b[(n-2)-i], c[(n-1)-i])
        #These two operations act as a sum gate; if a control bit is at                
        #the 1> state then the target bit b[(n-2)-i] is flipped
        qc.cx(c[(n-2)-i], b[(n-2)-i])
        qc.cx(a[(n-2)-i], b[(n-2)-i])

    #Measure qubits and store results in classical register cl
    for i in range(n+1):
        qc.measure(b[i], cl[i])

    trans_qc = transpile(qc, backend=device, optimization_level = 3)
    num_cnots = trans_qc.count_ops()['cx']
    #print("1 bit adder without assertion, size =",trans_qc.size(),"depth = ",trans_qc.depth(), "width = ",trans_qc.width(), "\n OPs =", trans_qc.count_ops(), "subcircuits =",trans_qc.num_tensor_factors())
    return trans_qc, num_cnots

In [7]:
def adder_assertion():
    first = "1"
    second = "1"

    l = len(first)
    l2 = len(second)
    if l > l2:
         n = l
    else:
         n = l2
    #Initializing the registers; two quantum registers with n bits each
    #1 more with n+1 bits, which will also hold the sum of the two #numbers
    #The classical register has n+1 bits, which is used to make the sum #readable
    a = QuantumRegister(n) #First number
    b = QuantumRegister(n+1) #Second number, then sum
    c = QuantumRegister(n) #Carry bits
    cl = ClassicalRegister(n+1) #Classical output
    #Combining all of them into one quantum circuit
    qc = QuantumCircuit(a, b, c, cl)

    #Setting up the registers using the values inputted
    for i in range(l):
        if first[i] == "1":
           qc.x(a[l - (i+1)]) #Flip the qubit from 0 to 1
    for i in range(l2):
       if second[i] == "1":
          qc.x(b[l2 - (i+1)]) #Flip the qubit from 0 to 1

    #Implementing a carry gate that is applied on all (c[i], a[i], b[i]) #with output fed to c[i+1]
    for i in range(n-1):
        qc.ccx(a[i], b[i], c[i+1])
        qc.cx(a[i], b[i])
        qc.ccx(c[i], b[i], c[i+1])
    #For the last iteration of the carry gate, instead of feeding the #result to c[n], we use b[n], which is why c has only n bits, with #c[n-1] being the last carry bit
    qc.ccx(a[n-1], b[n-1], b[n])
    qc.cx(a[n-1], b[n-1])
    qc.ccx(c[n-1], b[n-1], b[n])

    #Reversing the gate operation performed on b[n-1]
    qc.cx(c[n-1], b[n-1])
    #Reversing the gate operations performed during the carry gate implementations
    #This is done to ensure the sum gates are fed with the correct input bit states
    for i in range(n-1):
        qc.ccx(c[(n-2)-i], b[(n-2)-i], c[(n-1)-i])
        qc.cx(a[(n-2)-i], b[(n-2)-i])
        qc.ccx(a[(n-2)-i], b[(n-2)-i], c[(n-1)-i])
        #These two operations act as a sum gate; if a control bit is at                
        #the 1> state then the target bit b[(n-2)-i] is flipped
        qc.cx(c[(n-2)-i], b[(n-2)-i])
        qc.cx(a[(n-2)-i], b[(n-2)-i])


    #assertion
    qubitList = [b[1]] 
    success = classical_assertion(qc, qubitList, value = 1)
    if not success:
        print("Assertion Circuit Error")

    #Measure qubits and store results in classical register cl
    for i in range(n+1):
        qc.measure(b[i], cl[i])
    trans_qc = transpile(qc, backend=device, optimization_level = 3)
    num_cnots = trans_qc.count_ops()['cx']
    #print("1 bit adder without assertion, size =",trans_qc.size(),"depth = ",trans_qc.depth(), "width = ",trans_qc.width(), "\n OPs =", trans_qc.count_ops(), "subcircuits =",trans_qc.num_tensor_factors())
    return trans_qc, num_cnots

In [8]:
def adderExperiment():
    #This function run the transpiler multiple times and selects the mapping with fewest SWAP gates
    min_cnots = -1
    num_cnots_original = -1
    for i in range(10):
        trans_qc, num_cnots = adder()
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = adder()
        if num_cnots == min_cnots:
            print("\nnumber of CNOTs in circuit without assertion: ", num_cnots)
            num_cnots_original = num_cnots
            transQcList.append(trans_qc)
            break
            
    min_cnots = -1
    for i in range(20):
        trans_qc, num_cnots = adder_assertion()
        if min_cnots < 0:
            min_cnots = num_cnots
        elif (num_cnots < min_cnots) and (min_cnots >= 0):
            min_cnots = num_cnots
    while 1:
        trans_qc, num_cnots = adder_assertion()
        if num_cnots == min_cnots:
            print("number of CNOTs in circuit with assertion: ", num_cnots)
            percent = abs(1 - num_cnots_original/num_cnots) * 100
            print("    Percentage of the assertion circuit size in total size: ", percent, "%")
            if percent > 15:
                print("    Warning: assertion circuit size is comparable to circuit size, may lead to lower success rate.")
            transQcList.append(trans_qc)
            break

In [9]:
adderExperiment()


number of CNOTs in circuit without assertion:  11
number of CNOTs in circuit with assertion:  12
    Percentage of the assertion circuit size in total size:  8.333333333333337 %


In [10]:
shots = 8192
#execute the job on real device
job = execute(transQcList, backend=device, shots=shots)
job_monitor(job)
res = job.result()

Job Status: job has successfully run


Success rate of 1bit adder without assertion:

In [11]:
calcSuccessrate(res.get_counts(0), ['10'], 0)

total_count =  8192 success_count =  4767 success_rate =  58.19091796875 %


Success rate of 1bit adder with assertion:

In [12]:
calcSuccessrate(res.get_counts(1), ['0 10'], 1)

total_count =  6062 success_count =  4783 success_rate =  78.90135268888156 %
